In [3]:
! pip install librosa soundfile numpy sklearn pyaudio

     ------------------------------------ 214.3/214.3 kB 568.5 kB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 348.9 kB/s eta 0:00:00
     ------------------------------------ 164.0/164.0 kB 547.1 kB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 644.2 kB/s eta 0:00:00
     -------------------------------------- 56.3/56.3 kB 590.0 kB/s eta 0:00:00
     ---------------------------------------- 3.1/3.1 MB 1.1 MB/s eta 0:00:00
     -------------------------------------- 377.0/377.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 23.2/23.2 MB 1.8 MB/s eta 0:00:00
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23694 sha256=281b0d0968fa8aa8a036858b9b45f91ffff513ec48896055c08c0d1a129c000e
  Stored in directory: c:\users\aditya sandilya\appdata\local\pip\cache\wheels\e4\76\a4\cfb55573167a1f5b

In [1]:
import librosa
import soundfile
import glob
import os
import pickle

In [2]:
import numpy as np

In [3]:
def feature_extraction(file_name, power_spec,pitch, freq):
    with soundfile.SoundFile(file_name) as sound:
        a=sound.read(dtype='float32')
        rate=sound.samplerate
        if pitch:
            fourier_tran=np.abs(librosa.stft(a))
        results=np.array([])
        if power_spec:
            power=np.mean(librosa.feature.mfcc(y=a,sr=rate, n_mfcc=40).T, axis=0)
            results=np.hstack((results,power))
        if pitch:
            pitch=np.mean(librosa.feature.chroma_stft(S=fourier_tran,sr=rate).T,axis=0)
            results=np.hstack((results,pitch))
        if freq:
            freq=np.mean(librosa.feature.melspectrogram(a,sr=rate).T, axis=0)
            results=np.hstack((results,freq))
    return results


In [4]:
actual_emotions={   
    '01':'neutral',   
    '02':'calm',   
    '03':'happy',   
    '04':'sad',   
    '05':'angry',   
    '06':'fearful',   
    '07':'disgust',   
    '08':'surprised' } 
    #DataFlair - Emotions to observe 
emotions_to_observe=['fearful','neutral','calm', 'happy', 'angry', 'disgust','sad','surprised']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
def data_load(test_ratio=0.25):
    fea,e=[],[]
    for f in glob.glob(".\\ravdess-data\\Actor_*\\*.wav"):
        file_name=os.path.basename(f)
        #print(f)
        emotion=actual_emotions[file_name.split("-")[2]]
        #print(emotion)
        if emotion not in emotions_to_observe:
            continue
        feature=feature_extraction(f,power_spec=True, pitch=True, freq=True)
        fea.append(feature)
        e.append(emotion)
    return train_test_split(np.array(fea),e, test_size=test_ratio,random_state=6)

In [8]:
x_train,x_test,y_train,y_test=data_load(test_ratio=0.2)

C:\Users\ADITYA~1\AppData\Local\Temp/ipykernel_14428/1761490649.py:15: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  freq=np.mean(librosa.feature.melspectrogram(a,sr=rate).T, axis=0)
C:\Users\ADITYA~1\AppData\Local\Temp/ipykernel_14428/1761490649.py:15: FutureWarning: Pass y=[ 3.0517578e-05  3.0517578e-05  3.0517578e-05 ...  0.0000000e+00
  0.0000000e+00 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  freq=np.mean(librosa.feature.melspectrogram(a,sr=rate).T, axis=0)
C:\Users\ADITYA~1\AppData\Local\Temp/ipykernel_14428/1761490649.py:15: FutureWarning: Pass y=[3.0517578e-05 3.0517578e-05 0.0000000e+00 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  freq=np.mean(librosa.feature.melspectrogram(a,sr=rate).T, axis=0)
C:\Users\ADITY

In [9]:
x_test.shape[0]

288

In [10]:
x_train.shape[0]

1152

In [11]:
from sklearn.neural_network import MLPClassifier
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(350,), learning_rate='adaptive', max_iter=500)

In [12]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(350,),
              learning_rate='adaptive', max_iter=500)

In [13]:
y_pred=model.predict(x_test)

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5347222222222222